In [3]:
%%time
from fosforio.manager import get_conn_details_from_ds_name
from snowflake.snowpark.session import Session
from snowflake.ml.registry import Registry
import pandas as pd, re, json

CPU times: user 18 µs, sys: 8 µs, total: 26 µs
Wall time: 31 µs


In [2]:
def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

In [4]:
session = get_session('DIAMONDS',"fd0cf79c-8118-43a7-8f0b-059e8f78227a")

In [5]:
import requests
exp_id="8baa2fcf-6e7f-4bca-b2ae-776011ec4363"#"600b9f90-f431-4590-83c7-f38fe138dedc"
project_id="68bce134-6d7a-4e24-9591-86266438acf9"#"fd0cf79c-8118-43a7-8f0b-059e8f78227a"
executions_url = f"http://monitor-backend-service/monitor/api/v4/executions/search?objectId={exp_id}&projectId={project_id}&jobType=EXPERIMENT&profile=DETAILED"

temp_head = {
    "X-Auth-Userid": "aditya1.singh2@fosfor.com",
    "X-Auth-Username": "aditya1.singh2@fosfor.com",
    "X-Auth-Email": "aditya1.singh2@fosfor.com",
    "X-Project-Id": "68bce134-6d7a-4e24-9591-86266438acf9"
}

In [6]:
print(executions_url)

http://monitor-backend-service/monitor/api/v4/executions/search?objectId=8baa2fcf-6e7f-4bca-b2ae-776011ec4363&projectId=68bce134-6d7a-4e24-9591-86266438acf9&jobType=EXPERIMENT&profile=DETAILED


In [7]:
response = requests.get(executions_url, headers=temp_head)
print(response.json())
res = response.json()

{'totalRecordCount': 3, 'data': [{'jobinstanceId': 110212, 'objectId': '8baa2fcf-6e7f-4bca-b2ae-776011ec4363', 'projectId': '68bce134-6d7a-4e24-9591-86266438acf9', 'projectName': None, 'jobObjectName': 'New_Experiment_04', 'jobScheduleId': None, 'jobInstanceInputParameter': '{"input_params":"{\\"currentTime\\":\\"2024-07-11T05:24:29Z\\",\\"objectName\\":\\"New_Experiment_04\\",\\"lastRunDate\\":\\"\\",\\"instanceId\\":\\"110212\\",\\"lastSuccessfulRunDate\\":\\"\\",\\"userId\\":\\"aditya1.singh2@fosfor.com\\"}"}', 'jobType': 'EXPERIMENT', 'jobStatus': 'FAILED', 'finalStatus': 'FAILED', 'jobCreatedby': 'aditya1.singh2@fosfor.com', 'jobCreatedts': '2024-07-11T05:24:28.923+00:00', 'jobUpdatedby': 'aditya1.singh2@fosfor.com', 'jobUpdatedts': '2024-07-11T05:28:33.381+00:00', 'applicationId': '', 'executionId': None, 'runID': '110212', 'versionNumber': None, 'runEngine': 'SNOWFLAKE', 'attemptId': 110316, 'workflowId': None, 'jobConfig': '{"snowflake_query": "CALL RUN_EXPERIMENT({\'monitor_ru

In [8]:
_res_format = {
            "run_id": "",
            "algorithm_used": "",
            "run_status": "",
        }
monitor_resp = []

In [13]:
for data in res.get('data'):
    jobConfig=data.get('jobConfig')
    pattern = r"snowflake\.ml[^']*"
    match = re.search(pattern, jobConfig)
    if match:
        algorithm_used = match.group().rsplit('.', 1)
    print(data.get("runID"))
    _res_format.update({"run_id": f'"{data.get("runID")}"',
                        "run_status": data.get('jobStatus'),
                        "algorithm_used": algorithm_used[1] if match else None})
    res_data = _res_format
    print(res_data)
    monitor_resp.append(res_data)
print(monitor_resp)

110212
{'run_id': '"110212"', 'algorithm_used': 'XGBClassifier', 'run_status': 'FAILED'}
110213
{'run_id': '"110213"', 'algorithm_used': 'XGBClassifier', 'run_status': 'FAILED'}
110214
{'run_id': '"110214"', 'algorithm_used': 'XGBClassifier', 'run_status': 'SUCCESSFUL'}
[{'run_id': '"110214"', 'algorithm_used': 'XGBClassifier', 'run_status': 'SUCCESSFUL'}, {'run_id': '"110214"', 'algorithm_used': 'XGBClassifier', 'run_status': 'SUCCESSFUL'}, {'run_id': '"110214"', 'algorithm_used': 'XGBClassifier', 'run_status': 'SUCCESSFUL'}, {'run_id': '"110214"', 'algorithm_used': 'XGBClassifier', 'run_status': 'SUCCESSFUL'}, {'run_id': '"110214"', 'algorithm_used': 'XGBClassifier', 'run_status': 'SUCCESSFUL'}, {'run_id': '"110214"', 'algorithm_used': 'XGBClassifier', 'run_status': 'SUCCESSFUL'}, {'run_id': '"110214"', 'algorithm_used': 'XGBClassifier', 'run_status': 'SUCCESSFUL'}, {'run_id': '"110214"', 'algorithm_used': 'XGBClassifier', 'run_status': 'SUCCESSFUL'}, {'run_id': '"110214"', 'algorith

In [10]:
print(monitor_resp)

[{'run_id': '"110214"', 'algorithm_used': 'XGBClassifier', 'run_status': 'SUCCESSFUL'}, {'run_id': '"110214"', 'algorithm_used': 'XGBClassifier', 'run_status': 'SUCCESSFUL'}, {'run_id': '"110214"', 'algorithm_used': 'XGBClassifier', 'run_status': 'SUCCESSFUL'}]


In [ ]:
run_id = 109797
delete_run_url = f"http://monitor-backend-service/monitor/monitor/api/v1/delete/{run_id}"

In [ ]:
response = requests.delete(delete_run_url, headers=temp_head)
print(response.text)

In [ ]:
delete = sesseion.sql("DROP ")